# Choroid Plexus Segmentation Training via Auto3DSeg

In [1]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


MONAI version: 1.3.2
Numpy version: 1.26.4
Pytorch version: 2.4.0+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 59a7211070538586369afd4a01eca0a7fe2e742e
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.0
Nibabel version: 5.2.1
scikit-image version: 0.24.0
scipy version: 1.14.0
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.19.0+cu121
tqdm version: 4.66.4
lmdb version: 1.5.1
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.43.3
mlflow version: 2.15.0
pynrrd version: 1.0.0
clearml version: 1.16.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
reload_recursive(monai_training)
reload_recursive(mri_data)

from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [3]:
logger.remove()
logger.add(sys.stderr, level="DEBUG")

1

In [4]:
load_data = True

In [5]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "test"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair"]

## Prep the database

Get the data and labels organized

In [6]:
if load_data:
    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, 'r') as f:
        datalist = json.load(f)

    dataset = preprocess.parse_datalist(datalist_file, dataroot)

2024-11-05 20:20:03.943 | INFO     | monai_training.preprocess:parse_datalist:282 - Loading /home/srs-9/Projects/ms_mri/training_work_dirs/test/datalist.json
2024-11-05 20:20:03.944 | INFO     | monai_training.preprocess:parse_datalist:283 - /home/srs-9/Projects/ms_mri/training_work_dirs/test/datalist.json exists: True


In [7]:
subs = sorted([int(scan.subid) for scan in dataset])
print(subs)

[1010, 1011, 1019, 1033, 1065, 1080, 1109, 1119, 1163, 1188, 1191, 1234, 1259, 1265, 1272, 1280, 1293, 1321, 1355, 1437, 1486, 1498, 1518, 1547, 1548, 2081, 2083, 2144]


In [8]:
if not load_data:
    dataset_proc = DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses])
    dataset_proc.prepare_images(modalities)
    dataset_proc.prepare_labels(labels, suffix_list=["CH", "SRS", "ED", "DT"])

    dataset = dataset_proc.dataset
    dataset.sort()

In [9]:
if not load_data:
    dataset = training.assign_conditions(dataset, 0.2)
    dataset[0]

In [10]:
if not load_data:
    training_data = []
    test_data = []

    for scan in dataset:
        if scan.cond == 'tr':
            training_data.append({"image": scan.image, "label": scan.label})
        elif scan.cond == 'ts':
            test_data.append(scan.image)

## Review

In [11]:
def display_slices(scan):
    img1 = nib.load(scan.image_path)
    img2 = nib.load(scan.label_path)

    data1 = img1.get_fdata()[:,:,:]
    data2 = img2.get_fdata()

    slice_sums = np.sum(data2, axis=(0, 1))
    
    print(slice_sums)

    max_slice_index = np.argmax(slice_sums)
    print(f"Max slice: {max_slice_index}")

    slice1 = data1[:, :, max_slice_index]
    slice2 = data2[:, :, max_slice_index]    

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(slice1, cmap='gray')
    plt.title(f"Image 1 - Slice {max_slice_index}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(slice2, cmap='gray')
    plt.title(f"Image 2 - Slice {max_slice_index}")
    plt.axis('off')
    plt.show()


In [12]:
# display_slices(dataset[20])

In the original code, they include labels in the test data as well. Also they have a function that checks that there is nonzero number of voxels in the label

In [13]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]
if not load_data:
    train_data = []
    test_data = []
    for scan in dataset:
        if scan.cond == 'tr' and scan.has_label:
            train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
        elif scan.cond == 'ts' and scan.has_label():
            test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


    print(f"Train num total: {len(train_data)}")
    print(f"Test num: {len(test_data)}")

Create and save datalist

In [14]:
if not load_data:
    n_folds = 5
    datalist = {
        "testing": test_data,
        "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
    }

    if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

    # dataroot_dir = "/mnt/h"
    # if not os.path.isdir(dataroot_dir):
    #     os.makedirs(dataroot_dir)

    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, "w") as f:
        json.dump(datalist, f)

Load datalist

In [15]:
datalist['testing']

[{'image': '/mnt/h/3Tpioneer_bids/sub-ms1109/ses-20180303/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1109/ses-20180303/choroid_t1_flair.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1119/ses-20161010/choroid_t1_flair.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1163/ses-20180907/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1163/ses-20180907/choroid_t1_flair.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1191/ses-20190124/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1191/ses-20190124/choroid_t1_flair.nii.gz'},
 {'image': '/mnt/h/3Tpioneer_bids/sub-ms1355/ses-20210104/flair.t1.nii.gz',
  'label': '/mnt/h/3Tpioneer_bids/sub-ms1355/ses-20210104/choroid_t1_flair.nii.gz'}]

In [16]:
missing_images = []
missing_labels = []
for scan in dataset:
    if not scan.image_path.is_file():
        missing_images.append(scan.image_path)
    if not scan.label_path.is_file():
        missing_images.append(scan.label_path)
print(missing_images)
print(missing_labels)

OSError: [Errno 19] No such device: '/mnt/h/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.t1.nii.gz'

In [17]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2024-11-05 20:20:35,965 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/test
2024-11-05 20:20:35,970 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/test/datalist.json.
2024-11-05 20:20:35,971 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/test/datalist.json.
2024-11-05 20:20:35,984 - INFO - Using user defined command running prefix , will override other settings


In [18]:
max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [20]:
runner.run()

2024-11-05 20:25:44,637 - INFO - Running data analysis...
2024-11-05 20:25:44,640 - INFO - Found 1 GPUs for data analyzing!


File /home/srs-9/Projects/ms_mri/training_work_dirs/test/datastats.yaml already exists and will be overwritten.
100%|██████████| 23/23 [00:41<00:00,  1.79s/it]

2024-11-05 20:26:25,869 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2024-11-05 20:26:25,870 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/test/datastats.yaml.
2024-11-05 20:26:25,877 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/test/datastats_by_case.yaml, this may take a while.
2024-11-05 20:26:25,994 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 273kB/s]                              

2024-11-05 20:26:26,390 - INFO - Downloaded: /tmp/tmpno1emznr/algo_templates.tar.gz
2024-11-05 20:26:26,390 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpno1emznr/algo_templates.tar.gz.
2024-11-05 20:26:26,391 - INFO - Writing into directory: /home/srs-9/Projects/ms_mri/training_work_dirs/test.
2024-11-05 20:26:26,476 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0
2024-11-05 20:26:26,515 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_1
2024-11-05 20:26:26,553 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_2


2024-11-05 20:26:26,593 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_3
2024-11-05 20:26:26,636 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_4
2024-11-05 20:26:26,665 - INFO - ['python', '/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/configs/hyper_parameters.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/configs/network.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/configs/transforms_infer.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/configs/transforms_train.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/configs/transforms_validate.yaml'", '--num_epochs_per_validation=1', '--num_epochs=100', '--num_warmup_epochs=1']


/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2024-11-05 20:26:58,166 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2024-11-05 20:26:58,167 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/test/swinunetr_0/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2024/11/05 20:26:58 INFO mlflow.

KeyboardInterrupt: 

In [ ]:
scan_path = "/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/proc/lesion_index.t3m20-mni_reg.nii.gz"
img = nib.load(scan_path)
